In [1]:
import pandas as pd
import json
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive

drive.mount('/content/drive')

file_path = '/content/drive/My Drive/cell_phones_accessories.json'

Mounted at /content/drive


In [4]:
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]

df = pd.DataFrame(data)
df

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,4.0,No white background! It’s clear!,I bought this bc I thought it had the nice whi...,[{'small_image_url': 'https://images-na.ssl-im...,B08L6L3X1S,B08L6L3X1S,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1612044451196,0,True
1,5.0,Awesome! Great price! Works well!,Perfect. How pissed am I that I recently paid ...,[],B079BPGF6C,B079BPGF6C,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1534443517349,2,True
2,5.0,Worked but took an hour to install,Overall very happy with the end result. If you...,[{'small_image_url': 'https://m.media-amazon.c...,B088DR7Z5B,B0BBGGC8F2,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1629235304798,3,True
3,4.0,Decent,Lasted about 9 months then the lock button bro...,[{'small_image_url': 'https://images-na.ssl-im...,B07XRDHDNQ,B07XRDHDNQ,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1590470082910,0,True
4,5.0,LOVE IT!,LOVE THIS CASE! Works better than my expensive...,[],B00A8ZDL9Y,B00A8ZDL9Y,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1408994588000,0,True
...,...,...,...,...,...,...,...,...,...,...
999995,3.0,First of all just an FYI for those wondering y...,First of all just an FYI for those wondering y...,[],B00GGY8PAQ,B00GGY8PAQ,AH6G2UJ3CJ4PIO56IRAOF62GRWZQ,1419718065000,1,True
999996,5.0,Very good protection.,Looks good works well. Nothing else I expect.,[],B07H11F2GG,B07H11F2GG,AHQGXL5L3VG6EBZFAQP4R6IPOTGQ,1566098076662,0,True
999997,5.0,Good charging cord,Nice and long and braided coating looks like i...,[],B01CY1CUB4,B01CY1CUB4,AHQGXL5L3VG6EBZFAQP4R6IPOTGQ,1466052134000,0,True
999998,5.0,Perfect for iPhone! Love this!,This is perfect for my iPhone. I've bought oth...,[],B00GJ36AZG,B00GJ36AZG,AE2P2I3OV4K2N23GSAZADUCJ3RLQ,1435365140000,1,True


In [5]:
df

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,4.0,No white background! It’s clear!,I bought this bc I thought it had the nice whi...,[{'small_image_url': 'https://images-na.ssl-im...,B08L6L3X1S,B08L6L3X1S,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1612044451196,0,True
1,5.0,Awesome! Great price! Works well!,Perfect. How pissed am I that I recently paid ...,[],B079BPGF6C,B079BPGF6C,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1534443517349,2,True
2,5.0,Worked but took an hour to install,Overall very happy with the end result. If you...,[{'small_image_url': 'https://m.media-amazon.c...,B088DR7Z5B,B0BBGGC8F2,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1629235304798,3,True
3,4.0,Decent,Lasted about 9 months then the lock button bro...,[{'small_image_url': 'https://images-na.ssl-im...,B07XRDHDNQ,B07XRDHDNQ,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1590470082910,0,True
4,5.0,LOVE IT!,LOVE THIS CASE! Works better than my expensive...,[],B00A8ZDL9Y,B00A8ZDL9Y,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1408994588000,0,True
...,...,...,...,...,...,...,...,...,...,...
999995,3.0,First of all just an FYI for those wondering y...,First of all just an FYI for those wondering y...,[],B00GGY8PAQ,B00GGY8PAQ,AH6G2UJ3CJ4PIO56IRAOF62GRWZQ,1419718065000,1,True
999996,5.0,Very good protection.,Looks good works well. Nothing else I expect.,[],B07H11F2GG,B07H11F2GG,AHQGXL5L3VG6EBZFAQP4R6IPOTGQ,1566098076662,0,True
999997,5.0,Good charging cord,Nice and long and braided coating looks like i...,[],B01CY1CUB4,B01CY1CUB4,AHQGXL5L3VG6EBZFAQP4R6IPOTGQ,1466052134000,0,True
999998,5.0,Perfect for iPhone! Love this!,This is perfect for my iPhone. I've bought oth...,[],B00GJ36AZG,B00GJ36AZG,AE2P2I3OV4K2N23GSAZADUCJ3RLQ,1435365140000,1,True


In [6]:
df['verified_purchase'].value_counts()

,count
verified_purchase,
True,937676
False,62324


In [7]:
df = df.loc[df['verified_purchase'] != False]
df = df.reset_index(drop=True)
df

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,4.0,No white background! It’s clear!,I bought this bc I thought it had the nice whi...,[{'small_image_url': 'https://images-na.ssl-im...,B08L6L3X1S,B08L6L3X1S,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1612044451196,0,True
1,5.0,Awesome! Great price! Works well!,Perfect. How pissed am I that I recently paid ...,[],B079BPGF6C,B079BPGF6C,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,1534443517349,2,True
2,5.0,Worked but took an hour to install,Overall very happy with the end result. If you...,[{'small_image_url': 'https://m.media-amazon.c...,B088DR7Z5B,B0BBGGC8F2,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1629235304798,3,True
3,4.0,Decent,Lasted about 9 months then the lock button bro...,[{'small_image_url': 'https://images-na.ssl-im...,B07XRDHDNQ,B07XRDHDNQ,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1590470082910,0,True
4,5.0,LOVE IT!,LOVE THIS CASE! Works better than my expensive...,[],B00A8ZDL9Y,B00A8ZDL9Y,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,1408994588000,0,True
...,...,...,...,...,...,...,...,...,...,...
937671,5.0,Five Stars,Works well.,[],B00D3TAV5Y,B00D3TAV5Y,AHQSE5R5DERCA6VNSYAOTN3YAWDQ,1425904962000,0,True
937672,3.0,First of all just an FYI for those wondering y...,First of all just an FYI for those wondering y...,[],B00GGY8PAQ,B00GGY8PAQ,AH6G2UJ3CJ4PIO56IRAOF62GRWZQ,1419718065000,1,True
937673,5.0,Very good protection.,Looks good works well. Nothing else I expect.,[],B07H11F2GG,B07H11F2GG,AHQGXL5L3VG6EBZFAQP4R6IPOTGQ,1566098076662,0,True
937674,5.0,Good charging cord,Nice and long and braided coating looks like i...,[],B01CY1CUB4,B01CY1CUB4,AHQGXL5L3VG6EBZFAQP4R6IPOTGQ,1466052134000,0,True


In [8]:
df['verified_purchase'].value_counts()

,count
verified_purchase,
True,937676


In [9]:
'''
TF-IDF
'''

'\nTF-IDF\n'

In [10]:
review_vec = CountVectorizer(min_df=3, tokenizer=nltk.word_tokenize)
review_counts = review_vec.fit_transform(df['text'])
review_counts

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


<937676x43717 sparse matrix of type '<class 'numpy.int64'>'
	with 27343441 stored elements in Compressed Sparse Row format>

In [11]:
tfidf_transformer = TfidfTransformer()
review_tfidf = tfidf_transformer.fit_transform(review_counts)
review_tfidf

<937676x43717 sparse matrix of type '<class 'numpy.float64'>'
	with 27343441 stored elements in Compressed Sparse Row format>

In [12]:
'''
80-20 train test split approach
'''

'\n80-20 train test split approach\n'

In [13]:
x_train, x_test, y_train, y_test = train_test_split(review_tfidf, df['rating'], test_size=0.2)

In [14]:
lr_model = LogisticRegression(max_iter=1000)

lr_model.fit(x_train, y_train)
y_pred = lr_model.predict(x_test)

cf_matrix = confusion_matrix(y_test, y_pred)

print("---------------------------------------------------------------------------------------------------------------------")
print("80-20 split\n")
print("Accuracy:", accuracy_score(y_test, y_pred), "\n")
print(cf_matrix)
print("---------------------------------------------------------------------------------------------------------------------")

---------------------------------------------------------------------------------------------------------------------
80-20 split

Accuracy: 0.7348402440064841 

[[ 13835    935   1272    295   2666]
 [  4276   1099   2136    569   2388]
 [  2498    975   4080   2085   5269]
 [   820    242   2220   5059  15458]
 [  1261    200   1310   2852 113736]]
---------------------------------------------------------------------------------------------------------------------
